<a href="https://colab.research.google.com/github/milver/Experiments/blob/main/PWS_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Wunderground PWS Stations test

### Dependencies

In [21]:
from bs4 import BeautifulSoup as bs
import requests
import math
import re

### Functions

In [22]:
# @title
def scrape(station, units):
  USER_AGENT = "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36"
  LANGUAGE = "en-US,en;q=0.5"
  URL = "https://www.wunderground.com/dashboard/pws/"

  try:
      session = requests.Session()
      session.headers["User-Agent"] = USER_AGENT
      session.headers["Accept-Language"] = LANGUAGE
      session.headers["Content-Language"] = LANGUAGE
      html = session.get(URL + station["id"])

      soup = bs(html.text, "html.parser")
  except:
      print("error")
      return None
  data = {}


  data["LAST_UPDATED"] = soup.findAll(
      "span", attrs={"class": "ng-star-inserted"}, string=re.compile("updated")
  )[0].text
  print(soup.findAll(
      "span", attrs={"class": "ng-star-inserted"}, string=re.compile("updated")
  )[0].text)
  strings = data["LAST_UPDATED"].split()



  if (strings[0] == "(updated") and (strings[3] == "ago)"):
      value = int(strings[1])

      if (value >= 0) and (value <= 60):
          if strings[2][0:6] == "second":
              data["LAST_UPDATED"] = 1 / value

          elif strings[2][0:6] == "minute":
              data["LAST_UPDATED"] = 1 / (value * 60)

          elif strings[2][0:4] == "hour":
              if (value >= 0) and (value <= 24):
                  data["LAST_UPDATED"] = 1 / (value * 3600)

              else:
                  return None

          else:
              return None

      else:
          return None
  #print(strings)
  # Get Temperature
  if "temp" in station["parameters"]:
      # data["temp"] = soup.find("span", attrs={"class": "wu-value"})
      data["temp"] = soup.find(attrs={"class": "current-temp"})
      # print(data["temp"].text.replace('°', '').strip())

      data["temp"] = round(float(data["temp"].text.replace('°', '').strip()))

      if units["temp"] == "c":
          data["temp"] = round((data["temp"] - 32) * (5 / 9), 1)

  # Get Wind Speed
  if "wind_speed" in station["parameters"]:
      data["wind_speed"] = soup.findAll("span", attrs={"class": "wu-value"})
      #data["wind_speed"] = soup.findAll(attrs={"class": "weather__data weather__wind-gust"})
      print(data["wind_speed"])
      # the index is subject to change if they change the layout
      data["wind_speed"] = round(float(data["wind_speed"][8].text), 1)

      if units["speed"] == "kmph":
          data["wind_speed"] = round(data["wind_speed"] * 1.6, 1)

      elif units["speed"] == "mps":
          data["wind_speed"] = round(data["wind_speed"] * (4 / 9), 1)

  # Get Wind Gust
  if "wind_gust" in station["parameters"]:
      data["wind_gust"] = soup.findAll("span", attrs={"class": "wu-value"})
      data["wind_gust"] = round(float(data["wind_gust"][9].text), 1)

      if units["speed"] == "kmph":
          data["wind_gust"] = round(data["wind_gust"] * 1.6, 1)

      elif units["speed"] == "mps":
          data["wind_gust"] = round(data["wind_gust"] * (4 / 9), 1)


  # Get Precipitation Rate
  if "precip_rate" in station["parameters"]:
      data["precip_rate"] = soup.findAll("span", attrs={"class": "wu-value"})
      data["precip_rate"] = round(float(data["precip_rate"][11].text), 2)

      if units["precip"] == "mm":
          data["precip_rate"] = round(data["precip_rate"] * 25.4, 2)

  # Get Precipitation Total
  if "precip_total" in station["parameters"]:
      data["precip_total"] = soup.findAll("span", attrs={"class": "wu-value"})
      data["precip_total"] = round(float(data["precip_total"][14].text), 2)

      if units["precip"] == "mm":
          data["precip_total"] = round(data["precip_total"] * 25.4, 2)

  # Get Pressure
  if "pressure" in station["parameters"]:
      data["pressure"] = soup.findAll("span", attrs={"class": "wu-value"})
      data["pressure"] = round(float(data["pressure"][12].text), 2)

      if units["pressure"] == "hpa":
          data["pressure"] = round(data["pressure"] * 33.8637526, 2)

  # Get Humidity
  if "humidity" in station["parameters"]:
      data["humidity"] = soup.findAll("span", attrs={"class": "wu-value"})
      data["humidity"] = round(float(data["humidity"][13].text))

  # Get UV Index
  if "uv_index" in station["parameters"]:
      data["uv_index"] = soup.findAll("span", attrs={"class": "wu-value"})
      data["uv_index"] = round(float(data["uv_index"][15].text))

  # # Get Solar Radiation
  # if "radiation" in station["parameters"]:
  #     data["radiation"] = soup.findAll(
  #         "div", attrs={"class": "weather__text"}
  #     )
  #     strings = data["radiation"][-1].text.split()

  #     if strings[1][-8:-3] == "watts":
  #         data["radiation"] = round(float(strings[0]), 1)
  #     else:
  #         data["radiation"] = None

  station["data"] = data
  return station


### Computing weighted averages
Explanation:
Exponential Decay for Time Weighting:

An exponential decay function
weight
=
𝑒
−
𝜆
⋅
time
weight=e
−λ⋅time
  is used, where
𝜆
λ is a decay constant that can be adjusted based on how quickly you want the weight to decay with time.
Normalization of Time Weights:

The time weights are normalized so that their sum is 1.
Combining Weights:

The combined weight for each station is the product of its normalized time-based weight and its normalized distance-based weight.
This method provides a more robust way to account for the recency of the data while still considering the distance of the stations.

In [23]:
import math

def process(config):
    stations = []
    avg_values = {}

    # Distance-based weights
    distances = [1726, 3536, 3950]
    distance_weights = [1/d for d in distances]
    total_distance_weight = sum(distance_weights)
    normalized_distance_weights = [w/total_distance_weight for w in distance_weights]

    # Scrape data from stations
    for station in config["stations"]:
        stations.append(scrape(station, config["units"]))

    # Get average values, except wind bearing and wind direction
    for parameter in [
        "temp",
        "wind_speed",
        "wind_gust",
        "pressure",
        "humidity",
        "precip_rate",
        "precip_total",
        "uv_index",
        "radiation",
    ]:
        total_time_weight = 0
        time_weights = []

        # Calculate time-based weights using exponential decay
        for station in stations:
            if parameter in station["data"] and station["data"][parameter] is not None:
                # Example decay constant (lambda), adjust based on your preference
                decay_constant = 0.1
                time_since_update = station["data"]["LAST_UPDATED"]
                time_weight = math.exp(-decay_constant * time_since_update)
                time_weights.append(time_weight)
                total_time_weight += time_weight
            else:
                time_weights.append(0)

        avg_values[parameter] = 0
        combined_weights_sum = 0
        for i, station in enumerate(stations):
            if parameter in station["data"] and station["data"][parameter] is not None:
                # Normalize time-based weight
                time_weight = time_weights[i] / total_time_weight if total_time_weight > 0 else 0
                # Combine with distance-based weight
                combined_weight = time_weight * normalized_distance_weights[i]
                combined_weights_sum += combined_weight
                avg_values[parameter] += combined_weight * station["data"][parameter]

        if combined_weights_sum > 0:
            avg_values[parameter] = round(avg_values[parameter] / combined_weights_sum, 2)
        else:
            avg_values[parameter] = None  # No valid data for this parameter

    return avg_values



In [24]:
# @title
# def process(config):
#     stations = []
#     avg_values = {}
#     for station in config["stations"]:
#         stations.append(scrape(station, config["units"]))

#     # Get average values, except wind bearing and wind direction
#     for parameter in [
#         "temp",
#         "wind_speed",
#         "wind_gust",
#         "pressure",
#         "humidity",
#         "precip_rate",
#         "precip_total",
#         "uv_index",
#         "radiation",
#     ]:
#         last_updated_total = 0
#         for station in stations:
#             if parameter in station["data"] and station["data"][parameter] != None:
#                 last_updated_total += station["data"]["LAST_UPDATED"]

#         avg_values[parameter] = 0
#         for station in stations:
#             if parameter in station["data"] and station["data"][parameter] != None:
#                 last_updated_weight = (
#                     station["data"]["LAST_UPDATED"] / last_updated_total
#                 )
#                 avg_values[parameter] += (
#                     last_updated_weight * station["data"][parameter]
#                 )

#         avg_values[parameter] = round(avg_values[parameter], 2)


#     return avg_values

### Config

In [25]:
# @title
if __name__ == "__main__":
    config = {
        "units": {
            "temp": "F",
            "pressure": "hpa",
            "speed": "mph",
            "precip": "in",
        },
        "stations": [
            {
                "id": "KVAASHBU142",
                "parameters": [
                    "temp",
                    "wind_speed",
                    "wind_gust",
                    "wind_bearing",
                    "pressure",
                    "humidity",
                    "precip_rate",
                    "precip_total",
                    "uv_index",
                    "radiation",
                ],
            },
            {
                "id": "KVAASHBU115",
                "parameters": [
                    "temp",
                    "wind_speed",
                    "wind_gust",
                    "wind_bearing",
                    "pressure",
                    "humidity",
                    "precip_rate",
                    "precip_total",
                    "uv_index",
                    "radiation",
                ],
            },
            {
                "id": "KVABROAD56",
                "parameters": [
                    "temp",
                    "wind_speed",
                    "wind_gust",
                    "wind_bearing",
                    "pressure",
                    "humidity",
                    "precip_rate",
                    "precip_total",
                ],
            },
        ],
    }

### Run

In [26]:
for k, v in process(config).items():
    print(f"{k}\t{v}")

(updated 3 minutes ago)
[<span _ngcontent-sc183="" class="wu-value wu-value-to" style="color:;">58</span>, <span _ngcontent-sc183="" class="wu-value wu-value-to" style="color:;">69</span>, <span _ngcontent-sc183="" class="wu-value wu-value-to" style="color:;">81</span>, <span _ngcontent-sc183="" class="wu-value wu-value-to" style="color:;">60</span>, <span _ngcontent-sc183="" class="wu-value wu-value-to" style="color:;">76</span>, <span _ngcontent-sc183="" class="wu-value wu-value-to" style="color:;">66</span>, <span _ngcontent-sc183="" class="wu-value wu-value-to" style="color:;">60.8</span>, <span _ngcontent-sc183="" class="wu-value wu-value-to" style="color:;">60.8</span>, <span _ngcontent-sc183="" class="wu-value wu-value-to" style="color:;">0.0</span>, <span _ngcontent-sc183="" class="wu-value wu-value-to" style="color:;">0.0</span>, <span _ngcontent-sc183="" class="wu-value wu-value-to" style="color:;">60.5</span>, <span _ngcontent-sc183="" class="wu-value wu-value-to" style="col

In [27]:
print(config['stations'])

[{'id': 'KVAASHBU142', 'parameters': ['temp', 'wind_speed', 'wind_gust', 'wind_bearing', 'pressure', 'humidity', 'precip_rate', 'precip_total', 'uv_index', 'radiation'], 'data': {'LAST_UPDATED': 0.005555555555555556, 'temp': 61, 'wind_speed': 0.0, 'wind_gust': 0.0, 'precip_rate': 0.0, 'precip_total': 0.41, 'pressure': 1001.35, 'humidity': 99, 'uv_index': 0}}, {'id': 'KVAASHBU115', 'parameters': ['temp', 'wind_speed', 'wind_gust', 'wind_bearing', 'pressure', 'humidity', 'precip_rate', 'precip_total', 'uv_index', 'radiation'], 'data': {'LAST_UPDATED': 0.03225806451612903, 'temp': 62, 'wind_speed': 0.0, 'wind_gust': 0.0, 'precip_rate': 0.0, 'precip_total': 0.3, 'pressure': 1017.61, 'humidity': 95, 'uv_index': 0}}, {'id': 'KVABROAD56', 'parameters': ['temp', 'wind_speed', 'wind_gust', 'wind_bearing', 'pressure', 'humidity', 'precip_rate', 'precip_total'], 'data': {'LAST_UPDATED': 0.14285714285714285, 'temp': 62, 'wind_speed': 0.0, 'wind_gust': 0.0, 'precip_rate': 0.0, 'precip_total': 0.29,

### Station Data
*   X: 294ft
*   KVAASHBU142: 276ft - (DEM: 288); 1726 ft from X
*   KVAASHBU115: 344ft - (DEM: 339); 3536 ft from X
*   KVABROAD56: 351ft - (DEM: 348); 3950 ft from X



In [1]:
# Debug function
# import math

# def process(config):
#     stations = []
#     avg_values = {}

#     # Distance-based weights
#     distances = [1726, 3536, 3950]
#     distance_weights = [1/d for d in distances]
#     total_distance_weight = sum(distance_weights)
#     normalized_distance_weights = [w/total_distance_weight for w in distance_weights]

#     # Scrape data from stations
#     for station in config["stations"]:
#         stations.append(station)  # Use provided data directly for debugging

#     # Get average values, except wind bearing and wind direction
#     for parameter in [
#         "temp",
#         "wind_speed",
#         "wind_gust",
#         "pressure",
#         "humidity",
#         "precip_rate",
#         "precip_total",
#         "uv_index",
#         "radiation",
#     ]:
#         total_time_weight = 0
#         time_weights = []

#         # Calculate time-based weights using exponential decay
#         for station in stations:
#             if parameter in station["data"] and station["data"][parameter] is not None:
#                 # Example decay constant (lambda), adjust based on your preference
#                 decay_constant = 0.1
#                 time_since_update = station["data"]["LAST_UPDATED"]
#                 time_weight = math.exp(-decay_constant * time_since_update)
#                 time_weights.append(time_weight)
#                 total_time_weight += time_weight
#             else:
#                 time_weights.append(0)

#         avg_values[parameter] = 0
#         combined_weights_sum = 0
#         for i, station in enumerate(stations):
#             if parameter in station["data"] and station["data"][parameter] is not None:
#                 # Normalize time-based weight
#                 time_weight = time_weights[i] / total_time_weight if total_time_weight > 0 else 0
#                 # Combine with distance-based weight
#                 combined_weight = time_weight * normalized_distance_weights[i]
#                 combined_weights_sum += combined_weight
#                 avg_values[parameter] += combined_weight * station["data"][parameter]

#         if combined_weights_sum > 0:
#             avg_values[parameter] = round(avg_values[parameter] / combined_weights_sum, 2)
#         else:
#             avg_values[parameter] = None  # No valid data for this parameter

#     return avg_values

# # Example data
# config = {
#     "stations": [
#         {'id': 'KVAASHBU142', 'parameters': ['temp', 'wind_speed', 'wind_gust', 'wind_bearing', 'pressure', 'humidity', 'precip_rate', 'precip_total', 'uv_index', 'radiation'], 'data': {'LAST_UPDATED': 0.008333333333333333, 'temp': 61, 'wind_speed': 0.0, 'wind_gust': 0.0, 'precip_rate': 0.0, 'precip_total': 0.41, 'pressure': 1001.01, 'humidity': 99, 'uv_index': 0}},
#         {'id': 'KVAASHBU115', 'parameters': ['temp', 'wind_speed', 'wind_gust', 'wind_bearing', 'pressure', 'humidity', 'precip_rate', 'precip_total', 'uv_index', 'radiation'], 'data': {'LAST_UPDATED': 0.08333333333333333, 'temp': 62, 'wind_speed': 1.1, 'wind_gust': 2.2, 'precip_rate': 0.0, 'precip_total': 0.3, 'pressure': 1017.61, 'humidity': 95, 'uv_index': 0}},
#         {'id': 'KVABROAD56', 'parameters': ['temp', 'wind_speed', 'wind_gust', 'wind_bearing', 'pressure', 'humidity', 'precip_rate', 'precip_total'], 'data': {'LAST_UPDATED': 0.14285714285714285, 'temp': 62, 'wind_speed': 0.0, 'wind_gust': 0.0, 'precip_rate': 0.0, 'precip_total': 0.29, 'pressure': 1012.86, 'humidity': 94}}
#     ],
#     "units": "metric"  # or "imperial"
# }

# result = process(config)
# print(result)


{'temp': 61.48, 'wind_speed': 0.28, 'wind_gust': 0.56, 'pressure': 1007.88, 'humidity': 96.86, 'precip_rate': 0.0, 'precip_total': 0.36, 'uv_index': 0.0, 'radiation': None}
